### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [1]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [3]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [4]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [5]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [6]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [7]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [8]:
# una vez fiteado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [9]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [10]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [11]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [12]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [13]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [14]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [15]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([ 4811,  6635,  4253, ...,  1534, 10055,  4750])

In [16]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [17]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [18]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [19]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [20]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [21]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 1

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares.


### Punto 1

In [34]:
import random

random.seed(42)  
random_indices = random.sample(range(len(newsgroups_train.data)), 5)
random_documents = [newsgroups_train.data[idx] for idx in random_indices]

In [35]:
#calcular similaridad con el resto de los documentos
def calcular_similaridad(idx, X):
    cossim = cosine_similarity(X[idx], X)
    return cossim[0]

In [37]:
# calcula y analiza la similaridad para cada documento al azar
for idx, doc in enumerate(random_documents):
    print(f"Documento {idx + 1}:")
    print(doc[:200])  # se muestran solo los primeros 200 caracteres del documento
    print("")

    # calcula la similaridad con todos los documentos
    cossim = calcular_similaridad(random_indices[idx], X_train)

    # índices de los documentos más similares excluyendo el mismo documento
    most_similar_indices = np.argsort(cossim)[::-1][1:6]
    most_similar_texts = [newsgroups_train.data[i][:200] for i in most_similar_indices]

    print("Documentos más similares:")
    for i, sim_text in zip(most_similar_indices, most_similar_texts):
        print(f"Índice: {i}, Etiqueta: {newsgroups_train.target_names[newsgroups_train.target[i]]}")
        print(sim_text)
        print("")

    print("=" * 50)

Documento 1:
This is a general question for US readers:

How extensive is the playoff coverage down there?  In Canada, it is almost
impossible not to watch a series on TV (ie the only two series I have not had
an 

Documentos más similares:
Índice: 5064, Etiqueta: rec.sport.hockey

I only have one comment on this:  You call this a *classic* playoff year
and yet you don't include a Chicago-Detroit series.  C'mon, I'm a Boston
fan and I even realize that Chicago-Detroit games are

Índice: 9623, Etiqueta: talk.politics.mideast
Accounts of Anti-Armenian Human Right Violations in Azerbaijan #012
                 Prelude to Current Events in Nagorno-Karabakh

        +---------------------------------------------------------+


Índice: 10575, Etiqueta: sci.crypt

I am not an expert in the cryptography science, but some basic things
seem evident to me, things which this Clinton Clipper do not address.
The all pertain to opportunites for abuse, and conclusions 

Índice: 10836, Etiqueta: alt.at

### Punto 2

In [38]:
# Cargar los datos (nuevamente para tener los datos originales sin cambios)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

tfidf_vect = TfidfVectorizer()
X_train = tfidf_vect.fit_transform(newsgroups_train.data)
X_test = tfidf_vect.transform(newsgroups_test.data)

y_train = newsgroups_train.target
y_test = newsgroups_test.target

In [45]:
# entrenar y evaluar un clasificador
def train_and_evaluate(clf, X_train, y_train, X_test, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = f1_score(y_test, y_pred, average='macro')
    return score

# configuraciones para el vectorizador y los clasificadores
vectorizers = {
    'tfidf': TfidfVectorizer(),
    'tfidf_min_df_2': TfidfVectorizer(min_df=2),
    'tfidf_sublinear_tf': TfidfVectorizer(sublinear_tf=True),
}

classifiers = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB(),
}

for vec_name, vectorizer in vectorizers.items():
    print(f"Vectorizador: {vec_name}")
    X_train = vectorizer.fit_transform(newsgroups_train.data)
    X_test = vectorizer.transform(newsgroups_test.data)
    
    for clf_name, clf in classifiers.items():
        score = train_and_evaluate(clf, X_train, newsgroups_train.target, X_test, newsgroups_test.target)
        print(f"F1-score macro para {clf_name}: {score:.4f}")
    
    print("=" * 50)

Vectorizador: tfidf
F1-score macro para MultinomialNB: 0.5854
F1-score macro para ComplementNB: 0.6930
Vectorizador: tfidf_min_df_2
F1-score macro para MultinomialNB: 0.5970
F1-score macro para ComplementNB: 0.6935
Vectorizador: tfidf_sublinear_tf
F1-score macro para MultinomialNB: 0.5860
F1-score macro para ComplementNB: 0.6920


### Punto 3

In [46]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))

tfidf_vect = TfidfVectorizer()
X_train = tfidf_vect.fit_transform(newsgroups_train.data)

# trasposicion de la matriz documento-término para obtener una matriz término-documento
X_train_transposed = X_train.T

# se obtienen los términos (palabras) del vectorizador
terms = tfidf_vect.get_feature_names_out()

def calcular_similaridad_palabras(term, X):
    idx = np.where(terms == term)[0][0]  # Encontrar el índice del término
    cossim = cosine_similarity(X[idx], X)
    return cossim[0]


random.seed(120)  
random_indices = random.sample(range(len(terms)), 5)
random_words = [terms[idx] for idx in random_indices]

# se calcula y muestran las 5 palabras más similares para cada una de las palabras seleccionadas
for word in random_words:
    print(f"Palabra: {word}")

    # similaridad con todas las palabras
    cossim_words = calcular_similaridad_palabras(word, X_train_transposed)

    most_similar_indices = np.argsort(cossim_words)[::-1][1:6] # se excluye la palabra misma
    most_similar_words = [terms[i] for i in most_similar_indices]

    print("Palabras más similares:")
    for sim_word in most_similar_words:
        print(sim_word)

    print("=" * 50)


Palabra: occasional
Palabras más similares:
nu4
screwwing
boosters
fascinate
deliberation
Palabra: defensible
Palabras más similares:
welbon
zazen
sceptic
ricky
thirdbase
Palabra: burned
Palabras más similares:
_broken_
escpecially
preserving
widows
dripping
Palabra: slewed
Palabras más similares:
apologetic
fiamms
memorize
firebomb
blaring
Palabra: weird
Palabras más similares:
antidote
overstrain
foucault
longitudinal
_christians


### Conclusión:
Las palabras más similares para cada palabra seleccionada parecen ser bastante diversas y no siempre intuitivas. Esto puede ser debido a la manera en que el modelo de TF-IDF y la similitud coseno están calculando las similitudes basadas en la distribución y frecuencia de términos en los documentos del conjunto de datos.

Es importante destacar que la similaridad coseno considera la distribución relativa de las palabras en el conjunto de datos, por lo que las palabras más similares pueden no tener un significado directo o semántico obvio en algunos casos.

Para un análisis más profundo (a futuro) se podría ajustar el preprocesamiento de texto, y probar diferentes técnicas de vectorización (como Word2Vec o GloVe) o explorar modelos de lenguaje más avanzados.